# Project 2: Graph Mining
*Due Wednesday November 14th, 2018 at 11:59 pm*  
*Notebook Author: Koki Sasagawa*

Mining a large social network to uncover how well homophily can predict identity as well as the network structure. 

## Task 2:
Attribute Prediction - most of the nodes in the social network are provided with one or more attributes that can be drawn from different types. (e.g., age, occupation, musical preference, etc. ) Predict the probabilities of attributes for a set of completely unlabeled nodes

## Data
1. `labeled-vertices.train.tsv` & `labeled-vertices.dev.tsv` & `unlabeled-verticies.test.tsv` 
   - users with attributes formatted as the following: 

   > - **vertex1** T1:3 T7:1 T4:2
   > - **vertex2** T2:4
   > - **vertex3** T4:3 T3:1
   
   - Each value is specified as `AttributeType:Value`
   - Not every user will have their attributes listed 
   - Majority users should have at least 2 attribute set
2. `unlabeled-verticies.test.txt` - list of vertices that we will predict attributes and their values for

## Submission 

**Attribute prediction** should be a csv file with two columns: id and attr. 
The attr column should contain a space-deliminted list of the attributes you think the user with that id has. The file should have the following structure:

> id, attr
> 
> 123, T0:0 T1:1 

## Running the Notebook
This notebook requires the `nalgorithm.py` and `decorators.py` files.

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import time
import matplotlib.pyplot as plt
import seaborn as sns
from nalgorithm import candidate_node
from decorators import timer
%matplotlib inline

## 1. Create Graph

In [2]:
print("Creating network graph...")
start_time = time.perf_counter() 

with open("../raw_data/network.tsv", 'rb') as f:
    grph = nx.read_edgelist(path=f, delimiter='\t', encoding='utf8')

end_time = time.perf_counter() 
print("Network graph created. Process took {:.04f} seconds".format(end_time - start_time))

# Check that graph is of correct size
print("Number of edges: {}".format(grph.number_of_edges())) # There should be 30915267
print("Number of nodes: {}".format(grph.number_of_nodes())) # There should be 6626753

Creating network graph...
Network graph created. Process took 355.0810 seconds
Number of edges: 30915267
Number of nodes: 6626753


## 2. Load all files for attribute prediction 

In [3]:
@timer
def load_data(file_name):
    '''Load data in chunks and creates DataFrame'''
    
    chunks = pd.read_csv(file_name, 
                         delimiter='\t',
                         names=['id', 'attr'],
                         header=None,
                         chunksize=100000)

    output = pd.concat(chunks)
    
    return output

In [4]:
dev_set = load_data('../raw_data/labeled-vertices.dev.tsv')

print('Total number of nodes: {}'.format(dev_set.shape[0]))
dev_set.head()

Running load_data...
Finished in 0.3347s
Total number of nodes: 662675


,id,attr
0,2666403,T0:2 T1:99
1,2627940,T0:0 T1:26
2,4843136,T0:0 T1:26
3,5396835,T0:0 T1:1813
4,5438188,T0:1 T1:1733


In [5]:
train_set = load_data('../raw_data/labeled-vertices.train.tsv')

print('Total number of nodes: {}'.format(train_set.shape[0]))
train_set.head()

Running load_data...
Finished in 1.9837s
Total number of nodes: 5301403


,id,attr
0,5509623,T0:0 T1:0
1,6334893,T0:0 T1:1
2,1218900,T0:1 T1:2
3,3871398,T0:1 T1:2
4,3942361,T0:0 T1:3


**Important Note:** The following concatenation step was left out for the kaggle submission and all predictions were made using only the train_set. This error lead to many nodes with 'empty' attributes, leading to a lower performance score. Combining the two files improved the f1 score of the Adamic/Adar + Preferential Attachment predictions from 0.78475 to 0.81436. 

In [6]:
tot_set = pd.concat([train_set, dev_set]).reset_index(drop=True)
tot_set = tot_set.drop_duplicates(keep='first')

print('Total number of nodes: {}'.format(tot_set.shape[0]))
tot_set.head()

Total number of nodes: 5964078


,id,attr
0,5509623,T0:0 T1:0
1,6334893,T0:0 T1:1
2,1218900,T0:1 T1:2
3,3871398,T0:1 T1:2
4,3942361,T0:0 T1:3


In [7]:
print('Reading in test set...')
start_time = time.time()

test_set = []

with open('../raw_data/unlabeled-vertices.test.txt') as f:
    for line in f:
        test_set.append(line.rstrip())

end_time = time.time()
print("Test set loaded. Process took {:.04f} seconds".format(end_time - start_time))

print('Total number of nodes: {}'.format(len(test_set)))
test_set[:5]

Reading in test set...
Test set loaded. Process took 0.2055 seconds
Total number of nodes: 662675


['4546232', '3711008', '6394112', '5883774', '2843733']

## 3. Generate node pairs

Jaccard similarity and Adamic/Adar similarity both require nodes to have common neighbors, or the similarity score will be zero. Preferential attachment, however, can still be calculated as it is based on the idea that nodes will attach to nodes of higher degree. For nodes where Jaccard and Adamic/Adar fail to produce a similarity score, use the node suggested by rules of preferential attachment. 

In [8]:
candidate_node = timer(candidate_node)

### 3.a. Jaccard Similarity

In [30]:
jaccard_similarity_nodes = candidate_node(test_set=test_set, grph=grph, algo_type='jaccard')

Running candidate_node...
Finished in 9.4290s


### 3.b. Adamic/Adar Similarity

In [31]:
adamic_adar_nodes = candidate_node(test_set=test_set, grph=grph, algo_type='adamic/adar')

Running candidate_node...
Finished in 3.8526s


### 3.c. Preferential Attachment

In [32]:
preferential_attachment_nodes = candidate_node(test_set=test_set, grph=grph, algo_type='preferential attachment')

Running candidate_node...
Finished in 2.1646s


## 4. Select between Jaccard similarity, Adamic/Adar, or preferential attachment node pairs for attribute predictions 

### Round 1. Jaccard

In [33]:
data_1 = {'id': test_set,
          'attr': list(jaccard_similarity_nodes.values())}

### Round 2. Jaccard + Preferential Attachment

In [34]:
data_2 = {'id': test_set,
          'attr': list(jaccard_similarity_nodes.values()), 
          'attr2': list(preferential_attachment_nodes.values())}

### Round 3. Adamic/Adar 

In [35]:
data_3 = {'id': test_set,
          'attr': list(adamic_adar_nodes.values())}

### Round 4. Adamic/Adar + Preferential Attachment

In [36]:
data_4 = {'id': test_set,
          'attr': list(adamic_adar_nodes.values()), 
          'attr2': list(preferential_attachment_nodes.values())}

### Select prediction of interest

In [37]:
predictions = pd.DataFrame(data_4)

jaccard_similarity_node = None
adamic_adar_nodes = None
preferential_attachment_nodes = None

### Combine predictions from Jaccard or Adamic/Adar with preferential attachment

In [38]:
if len(predictions.columns) == 3: 
    # Number of times where candidate node does not exist
    zeros = predictions.shape[0] - np.count_nonzero(predictions['attr'])
    print("Missing candidate nodes: {}".format(zeros))
    
    print('Replacing 0\'s with nodes from preferential attachment...')
    # Replace 0's with nodes selected by preferential attachment
    predictions['attr'] = np.where(predictions['attr'] == 0, predictions['attr2'], predictions['attr'])
    zeros = predictions.shape[0] - np.count_nonzero(predictions['attr'])
    print("Missing candidate nodes: {}".format(zeros))
    
    predictions.drop(['attr2'], axis=1, inplace=True)

Missing candidate nodes: 58932
Replacing 0's with nodes from preferential attachment...
Missing candidate nodes: 0


## 5. Obtain attributes for nodes

In [39]:
print("There are {} rows in the prediction dataframe".format(predictions.shape[0]))
predictions.head()

There are 662675 rows in the prediction dataframe


,id,attr
0,4546232,2494614
1,3711008,2444912
2,6394112,6223074
3,5883774,4485305
4,2843733,3931905


In [40]:
# Check dtype
print('dtype of id in tot_set is {}'.format(type(tot_set['id'][0])))
print('dtype of id in predictions is {}'.format(type(predictions['attr'][0])))

dtype of id in tot_set is <class 'numpy.int64'>
dtype of id in predictions is <class 'str'>


In [41]:
# The values in the prediction dataframe are currently str type
# convert them to int64 to allow merge with tot_set
predictions = predictions.astype(dtype=np.int64, copy=True)
print('dtype of id in predictions is {}'.format(type(predictions['attr'][0])))

dtype of id in predictions is <class 'numpy.int64'>


In [42]:
results = predictions.merge(tot_set, left_on='attr', right_on='id', how='left')
results.head()

,id_x,attr_x,id_y,attr_y
0,4546232,2494614,2494614.0,T0:0 T1:1766
1,3711008,2444912,2444912.0,T0:0 T1:1762
2,6394112,6223074,6223074.0,T0:0 T1:1914 T8:0
3,5883774,4485305,4485305.0,T0:0 T1:944
4,2843733,3931905,3931905.0,T0:0 T1:538


In [43]:
print('nan values:\n{}'.format(results.isna().sum()))

nan values:
id_x          0
attr_x        0
id_y      32765
attr_y    32765
dtype: int64


There are 32765 nodes that we do not know attributes for. Our current method is unable to predict attributes for these nodes. 

In [44]:
results.drop(['attr_x', 'id_y'], inplace=True, axis=1)
results.columns = ['id', 'attr']
results.head()

,id,attr
0,4546232,T0:0 T1:1766
1,3711008,T0:0 T1:1762
2,6394112,T0:0 T1:1914 T8:0
3,5883774,T0:0 T1:944
4,2843733,T0:0 T1:538


In [45]:
print('Results contain {} nodes'.format(results.shape[0]))

Results contain 662675 nodes


## 6. Save attribute predictions as csv file

In [46]:
results.to_csv('../output/attribute_predictions.csv', index=False)

For the particular dataset, homophily does not appear to hold well. Again, homophily in a social context explains that people who share similarities in socially significant ways are more likely to be linked. A subgraph of the original graph was created using the neighboring nodes of a randomly selected node. For each node in the subgraph, we calculated the similarity scores with its neighbors (if any), and returned the highest similarity score which was plotted on a histogram. It appears that most of the similarity scores were on the lower end (right-skewed) except a few with high similarity scores, and most nodes did not share the same attributes as only 11 of 71 nodes did. Of nodes with matching attributes, it seems the majority had low similarity scores as well (right-skewed). 

# Future Approaches 

For the next steps, I would like to revise my current approach for attribute predictions and see how the algorithm can be improved by taking into account attribute trends. From observing the data, I realized that out of the three classes of attributes (T0, T1, and T8), T0 and T1 were far more prevalent than T8. Perhaps, in real life social network, T0 and T1 are attributes much more common among users, whereas T8 is rarer and much more specific. For example, in LinkedIn, T0 could be a user’s educational background which, whereas T8 could be a professional certification a user has obtained. On a business professional network, ones educational background is a common attribute included in many profiles, but much fewer individuals may have certifications listed, such as being certified doctor, lifeguard, lawyer, etc. 
 
For more common classes of attributes (T0, T1), I could develop an algorithm that counts and assigns the most common attribute shared among all nearest neighbors. For example, in a given scenario, if T0:1 was more common than T0:2 out of all nearest neighbors, then inherit T0:1. 

For rarer classes of attributes (T8), I would use my current method of inheriting the attribute of the most similar neighbor as calculated by the Adamic/Adar algorithm. In cases where multiple nodes have the same similarity score and also have different variations of the rare attribute, I could either select the winning attribute randomly or by frequency (e.g., T8:213 might be more common than T8:121, so inherit T8:213).

Additionally, I would like to explore other algorithms, such as treating attributes as itemsets which can be used to build association rules (e.g, If T0:12 and T1:334 are present, then T8:123 is also likely to be present).